In [ ]:
import pandas as pd
import datetime
from matplotlib import pyplot as plt
from textwrap import wrap
import numpy as np
import geopandas as gpd
from src.utils.styling import hide_and_move_axis
from pathlib import Path
import matplotlib
import hydra
import os

In [ ]:
os.chdir('..')

OUTPUT_FOLDER = 'output/{0}'.format(datetime.datetime.now())
Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

hydra.initialize(version_base=None, config_path="../config/")
CONFIG = hydra.compose(config_name="main.yaml")

In [ ]:
wording = {
    'q49': 'For the last four weeks I have been happy and in good spirits.',
    'q50': 'In the last four weeks I have felt calm and relaxed.',
    'q54': 'For the past four weeks, I have felt energetic and active.',
    'q55': 'For the past four weeks, I have felt fresh and rested when I wake up.',
    'q56': 'For the past four weeks, my daily life has been full of things that interest me.',
    'total_wellbeing': 'Average wellbeing'
}

In [ ]:
data = pd.read_feather(Path(CONFIG.data.processed) / CONFIG.data.filenames.merged_data)

In [ ]:
label = {'v9total': 'Steps', 
         'v65total': 'Resting heart rate', 
         'v43total': 'Sleep duration', 
         'v52total': 'Sleep onset', 
         'v53total': 'Sleep offset', 
         'midsleeptotal': 'Midsleep', 
         'v43difference': 'Difference in Sleep Duration WE-WD',
         'v52difference': 'Difference in Sleep Onset  WE-WD',
         'v53difference': 'Difference in Sleep Offset WE-WD',
         'v9difference': 'Difference in steps WE-WD',
         'v65difference': 'Difference in RHR WE-WD',
         'social_jetlag': 'Social Jetlag',
        }


def plot_survey_response_per_vitals(vital_key):
    f, axarr = plt.subplots(2, 3, sharex=True, figsize=(7,5))

    bins = 40
    question_keys = ['q49', 'q50', 'q54', 'q55', 'q56', 'total_wellbeing']
    marker = 0

    for question_key, ax in zip(question_keys, axarr.flatten()):
        df = data[[vital_key, question_key]].dropna()

        df['bins'] = pd.cut(df[vital_key], bins)
        df.bins = df['bins'].apply(lambda x: x.left + 0.5 * (x.right - x.left))

        df = df.groupby('bins', observed=True)[question_key].agg(['mean', 'count', 'std'])
        df['err'] = 1.96 * df['std'] / np.sqrt(df['count'])
        df = df[df['count'] > 50]

        ax.errorbar(df.index, df['mean'], yerr=df['err'], fmt='o', markersize=5)
        ax.set_title('\n'.join(wrap(wording[question_key], 30)), size=10)

        if (vital_key == 'social_jetlag') or ('difference' in vital_key):
            if marker != 'None':
                ax.axvline(marker, c='k', ls=':')

        hide_and_move_axis(ax)
        
    for ax in axarr[1]:
        ax.set_xlabel(label[vital_key])

    plt.tight_layout()
    plt.savefig(f'{OUTPUT_FOLDER}/survey_response_per_{vital_key}.jpg', dpi=400)
    
    
for vital_key in label.keys():
    plot_survey_response_per_vitals(vital_key)